# More on metadata of arrays
## Array interface
Approach to the interface of array using private attribute:

In [41]:
import numpy as np
arr = np.random.randint(2**32, size=(2, 3), dtype=np.uint32)
arr

array([[ 658047583, 2399717312, 2682325474],
       [3789451691, 2585146673,  388629288]], dtype=uint32)

In [42]:
arr.__array_interface__

{'data': (94920605948720, False),
 'strides': None,
 'descr': [('', '<u4')],
 'typestr': '<u4',
 'shape': (2, 3),
 'version': 3}

## Pointers

Approach to: 
* a pointer to the data-area storing the array contents (it must point to the first element of data)
* a read-only flag (true means the data area is read-only)

In [43]:
pointer, read_only = arr.__array_interface__['data']
pointer, read_only

(94920605948720, False)

In [44]:
arr.nbytes

24

In order to see what's inside the data storage area, approach to its bytes, 4 by 4:

In [45]:
from ctypes import string_at
bytes = np.array([n for n in string_at(pointer, arr.nbytes)]).reshape(-1,4)
bytes

array([[ 95,   2,  57,  39],
       [192, 199,   8, 143],
       [226,   9, 225, 159],
       [171, 113, 222, 225],
       [ 49,  53,  22, 154],
       [ 40,   3,  42,  23]])

Extract initial data from bytes:

In [50]:
stream = [a + 2**8*b + 2**16*c + 2**24*d for a, b, c, d in bytes]
stream

[658047583, 2399717312, 2682325474, 3789451691, 2585146673, 388629288]

Create a view of initial array:

In [51]:
shape = arr.__array_interface__['shape']
shape

(2, 3)

In [52]:
np.reshape(stream, shape)

array([[ 658047583, 2399717312, 2682325474],
       [3789451691, 2585146673,  388629288]])

Make sure it's the same with initial array:

In [54]:
np.array_equal(arr, np.reshape(stream, shape))

True

## Read-only flag
Take a look how to make `numpy` array immutable

In [73]:
arr = np.ones((2, 2), dtype=int)
arr

array([[1, 1],
       [1, 1]])

In [74]:
arr[0, 1] = -9999
arr

array([[    1, -9999],
       [    1,     1]])

In [75]:
print(arr.__array_interface__['data'])
arr.setflags(write = False)
print(arr.__array_interface__['data'])

(94920600347216, False)
(94920600347216, True)


In [76]:
arr[0, 1] = 1
arr

ValueError: assignment destination is read-only

## Strides
This is old, not safe method

In [94]:
arr = np.arange(24).reshape(4, 6)
arr

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23]])

In [95]:
arr.strides

(48, 8)

Create a specific view of array according to these rules:  
* `strides` tells how many bytes a pointer needs to move along each axis while creating a view of plain array  
* `shape` tells the total number of moves along each axis  
* plain array is: `[0, 1, ..., 22, 23]`  

In [138]:
# move 1 element along axis=0 and 6 elements along axis=1; take 6 elements on axis=0 and 4 elements on axis=1
np.lib.stride_tricks.as_strided(arr, shape=(6, 4), strides=(arr.itemsize, 6 * arr.itemsize))

array([[ 0,  6, 12, 18],
       [ 1,  7, 13, 19],
       [ 2,  8, 14, 20],
       [ 3,  9, 15, 21],
       [ 4, 10, 16, 22],
       [ 5, 11, 17, 23]])

In [139]:
# move 2 elements along axis=0 and 2 elements along axis=1; take 10 elements on axis=0 and 3 elements on axis=1
np.lib.stride_tricks.as_strided(arr, shape=(10, 3), strides=(2*arr.itemsize, 2*arr.itemsize))

array([[ 0,  2,  4],
       [ 2,  4,  6],
       [ 4,  6,  8],
       [ 6,  8, 10],
       [ 8, 10, 12],
       [10, 12, 14],
       [12, 14, 16],
       [14, 16, 18],
       [16, 18, 20],
       [18, 20, 22]])

## Windows

In numpy version >= 1.20.0 (Jan 30, 2021) you wouldn't need to calculate strides manually if you want to create a sliding window view into the array with the given window shape

In [3]:
x = np.arange(6)
print(x)
print(np.lib.stride_tricks.sliding_window_view(x, 3))

[0 1 2 3 4 5]
[[0 1 2]
 [1 2 3]
 [2 3 4]
 [3 4 5]]


In [4]:
x = np.arange(15).reshape(3,5)
print(x)
print(np.lib.stride_tricks.sliding_window_view(x, (2, 2)))

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]
[[[[ 0  1]
   [ 5  6]]

  [[ 1  2]
   [ 6  7]]

  [[ 2  3]
   [ 7  8]]

  [[ 3  4]
   [ 8  9]]]


 [[[ 5  6]
   [10 11]]

  [[ 6  7]
   [11 12]]

  [[ 7  8]
   [12 13]]

  [[ 8  9]
   [13 14]]]]
